In [1]:
import numpy as np
from sklearn import svm
import dv
import cv2
import time

In [2]:
labels = np.genfromtxt('user03_fluorescent_led_labels.csv', delimiter=',', skip_header=1, dtype=int)
print(labels)

[[        1  42148585  48082899]
 [        2  48613803  55081191]
 [        3  55708624  62594300]
 [        4  63720462  71458804]
 [        5  71684037  78368585]
 [        6  79197146  85214069]
 [        7  85535829  91504488]
 [        8  92067569  98148844]
 [        8  98486692 104598585]
 [        9 105163224 110318585]
 [       10 111180147 117518830]
 [       11 117937119 121338585]]


In [3]:
# Return the start time of the given class index.
def start(classIndex):
    return labels[classIndex][1]
 
# Return the end time of the given class index.
def end(classIndex):
    return labels[classIndex][2]

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime):
    for i in range(10):
        if (currentTime >= start(i) and currentTime <= end(i)):
            return i
    return -1

In [4]:
# Test the setGesture function

tests = [[42148585, 0], # Gesture 1 has just started 
        [42148584, -1], # Gesture 1 is about to started
        [48082899, 0],  # Gesture 1 is about to end
        [48082900, -1], # Gesture 1 has just ended
         
        [48613803, 1],  # Gesture 2 just started
        [55708624, 2],
        [63720462, 3],
        [121338586, -1],]

for i in tests:
    if (setGesture(i[0]) == i[1]):
        print("Pass")
    else:
        print(setGesture(i[0]), i[1])

Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass


In [5]:
X = []
Y = []
offset = 0
timestamp = 0

with dv.AedatFile("dvSave-2022_10_11_20_53_26.aedat4") as f: 
    for frame in f['frames']:    
        # Set the offset on the first frame.
        if (offset == 0):
            offset = frame.timestamp

        gesture = setGesture((frame.timestamp - offset))
        image = frame.image
        
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        # org
        org = (5, 30)

        # fontScale
        fontScale = 1

        # Blue color in BGR
        color = (255, 0, 0)

        # Line thickness of 2 px
        thickness = 2
        image = cv2.putText(image, str(gesture), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        
        cv2.imshow('out',image)
        cv2.waitKey(1)
        
        if (gesture == -1):
            continue
        else:
            Y.append(gesture)

            matrix = frame.image.flatten()
            X.append(matrix)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to target thread (0x561f846e5860)

QObject::moveToThread: Current thread (0x561f846e5860) is not the object's thread (0x561f84e8da40).
Cannot move to tar

In [6]:
print(len(X))
print(len(Y))

1058
1058


In [7]:
clf = svm.SVC()
clf.fit(X, Y)

SVC()

In [8]:
clf.predict([X[1780]])

IndexError: list index out of range